<a href="https://colab.research.google.com/github/hariprasanth-22/Instruction-Tuning-Falcon-with-LoRA/blob/main/LoRA_Falcon_Q%26A_Fine_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets peft accelerate  bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,TrainingArguments,Trainer
from peft import LoraConfig, get_peft_model,TaskType
from datasets import Dataset


In [ ]:
data={
    "text":[
        "### Question: What is the capital of France?\n### Answer: Paris",
        "### Question: What is the largest planet in our solar system?\n### Answer: Jupiter",
        "### Question: What is the square root of 25?\n### Answer: 5"
    ]
}
dataset=Dataset.from_dict(data)

In [ ]:
model_name="tiiuae/falcon-rw-1b"
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
def tokenize(batch):
  tokens=tokenizer(batch["text"],truncation=True,padding="max_length",max_length=128)
  tokens["labels"]=tokens["input_ids"].copy()
  return tokens

In [ ]:
tokenized_dataset=dataset.map(tokenize,batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",

)

config.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM

)

In [ ]:
model=get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [ ]:
training_args=TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    learning_rate=2e-4,
    logging_steps=1,
    save_strategy="no",
    num_train_epochs=3,
    report_to="none"


)

In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,

)
trainer.train()

/tmp/ipython-input-11-883548521.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,5.645500
2,4.876400
3,4.161900
4,3.846900
5,3.894000
6,3.027900
7,2.958400
8,2.483200
9,2.642800


TrainOutput(global_step=9, training_loss=3.72633335325453, metrics={'train_runtime': 9.3283, 'train_samples_per_second': 0.965, 'train_steps_per_second': 0.965, 'total_flos': 8364732973056.0, 'train_loss': 3.72633335325453, 'epoch': 3.0})

In [ ]:
model.save_pretrained("lora-finetuned")
tokenizer.save_pretrained("lora-finetuned")

('lora-finetuned/tokenizer_config.json',
 'lora-finetuned/special_tokens_map.json',
 'lora-finetuned/vocab.json',
 'lora-finetuned/merges.txt',
 'lora-finetuned/added_tokens.json',
 'lora-finetuned/tokenizer.json')